<a href="https://colab.research.google.com/github/blid11/Group9/blob/Brett/Copy_of_G9_PartC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Changes made in this Colab notebook can be saved with Control/Command S and do not need to be pushed to Github. 

In [ ]:

# Import libaries needed
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# made a change in colab

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

%cd gdrive/My Drive/G9A2/G9
 
#%cd gdrive/My Drive/G9A2
# I made a folder on my google drive that has the G9 Methylation data and the binfpy library
#this code automatically loads it in
# you can do the same with your google drive. 

Mounted at /content/gdrive
/content/gdrive/My Drive/G9A2/G9


In [ ]:
#set styles of plots - taken from Week 6 notebook
sns.set_theme(style="ticks")

sns.set(rc={'figure.figsize': (5, 5), 'font.family': 'sans-serif',
            'font.sans-serif': 'Arial', 'font.size': 10, 'axes.titlesize': 16,
           'axes.titleweight': 700}, style='ticks')
sns.set_palette("Set2")

# Data Preparation


In [ ]:
!ls

G9_liver_dna-meth.csv  G9_liver_gene-expr.csv


In [ ]:
# load the data 
liver_meth_dna = pd.read_csv('G9_liver_dna-meth.csv')
liver_gene_expr = pd.read_csv('G9_liver_gene-expr.csv')


In [ ]:
# view the data: 
liver_meth_dna.head(n = 5)

,Unnamed: 0,Label,cg00005740,cg00012529,cg00013196,cg00019366,cg00028598,cg00030627,cg00033551,cg00047815,...,rs7660805,rs7746156,rs798149,rs845016,rs877309,rs9292570,rs9363764,rs939290,rs951295,rs966367
0,TCGA.2V.A95S.01,Primary Tumor,0.601838,0.374335,0.300235,0.259656,0.311692,0.331413,0.963102,0.586768,...,0.944546,0.959710,0.969422,0.368036,0.513791,0.509061,0.541551,0.562851,0.495167,0.048576
1,TCGA.2Y.A9GS.01,Primary Tumor,0.399472,0.395836,0.395846,0.380457,0.273681,0.446539,0.952035,0.380159,...,0.334419,0.459250,0.971834,0.933756,0.521390,0.020679,0.573997,0.960284,0.037918,0.473765
2,TCGA.2Y.A9GT.01,Primary Tumor,0.438239,0.640740,0.477210,0.519875,0.377109,0.450508,0.604261,0.558608,...,0.511502,0.462957,0.018341,0.951748,0.544744,0.025068,0.558837,0.966630,0.968370,0.948648
3,TCGA.2Y.A9GU.01,Primary Tumor,0.913263,0.969171,0.944687,0.573213,0.944506,0.934732,0.909746,0.883221,...,0.540377,0.487190,0.015734,0.945182,0.020356,0.521952,0.939739,0.966117,0.027995,0.949593
4,TCGA.2Y.A9GV.01,Primary Tumor,0.648241,0.805019,0.560080,0.914805,0.389027,0.641507,0.218462,0.559733,...,0.948858,0.957963,0.016105,0.940811,0.569243,0.967371,0.518856,0.523212,0.506449,0.037212


In [ ]:
liver_gene_expr.head(n= 5)
#the first column is sample
#the second column is the Label 
# the third column onwards are Data (expression values for given gene)

,Unnamed: 0,Label,X..100130426,X..100133144,X..100134869,X..10357,X..10431,X..136542,X..155060,X..26823,...,ZXDA.7789,ZXDB.158586,ZXDC.79364,ZYG11A.440590,ZYG11B.79699,ZYX.7791,ZZEF1.23140,ZZZ3.26009,psiTPTE22.387590,tAKR.389932
0,TCGA.2V.A95S.01,Primary Tumor,0.0,1.5051,3.7074,90.1124,1017.1038,0.0,141.3911,0.6516,...,24.7597,273.6602,794.2662,18.2440,499.1041,3172.5037,890.0472,510.1808,3.9094,6.5157
1,TCGA.2Y.A9GS.01,Primary Tumor,0.0,26.4120,2.6663,71.0054,639.2311,0.0,122.7206,1.4786,...,68.5067,632.8241,1153.7703,71.4638,1000.4929,5301.1336,755.5446,860.5224,6.4071,482.9966
2,TCGA.2Y.A9GT.01,Primary Tumor,0.0,0.0000,4.4833,95.5122,742.4344,0.0,95.0460,1.7933,...,46.6263,1219.4575,1133.3782,12.5532,1289.3970,3219.0092,860.7935,523.6494,14.3466,83.3894
3,TCGA.2Y.A9GU.01,Primary Tumor,0.0,5.7222,5.1216,61.6679,1186.9807,0.0,280.2709,0.8341,...,18.3511,285.2758,1150.2786,9.1755,941.7437,3092.9899,1339.6283,343.6655,2.5024,2.5024
4,TCGA.2Y.A9GV.01,Primary Tumor,0.0,11.4975,5.4230,104.4670,878.1726,0.0,282.5719,0.0000,...,41.4552,999.1540,1631.9797,4.2301,1380.7107,2902.7073,575.2961,665.8206,2.5381,119.2893


In [ ]:
# convert labels to 1 or 0 
bool_tum_expr = [1 if value == 'Primary Tumor' else 0 for value in liver_gene_expr['Label'].values]
bool_tum_meth = [1 if value == 'Primary Tumor' else 0 for value in liver_meth_dna['Label'].values]

# DO NOT add the numeric label values to the dataframes yet 
# liver_gene_expr['BinaryLabel'] = bool_tum_expr
# liver_meth_dna['BinaryLabel'] = bool_tum_meth
# confirm that the changes were added
# liver_gene_expr.head(n= 2)
# liver_meth_dna.head(n = 2)

# I'm not sure if we need to convert the gene expression/methylation to numeric

will be creating two models: one for gene expression and one for methylation
so will not merge the two datasets 
we can use the same model architecture and train it on different data

standardizing and log transforming data relatively standard for gene expression data
and probably for methylation data as well 
will do this prior to training the two PCA models

After PCA has reduced dimsensionality, we can convert the output values to between 0 and 1 for a neural network or SVM if necessary. 

In [ ]:
# standardizing and log transform data prior to PCA
# for each numeric value column in the dataframem, normalize 
# and log tranform the values

col_names_exp = liver_gene_expr.columns
col_names_meth = liver_meth_dna.columns

#replace Inf and Nan with 0 
liver_gene_expr.replace([np.inf, -np.inf], 0, inplace=True)
liver_meth_dna.replace([np.inf,-np.inf], 0, inplace=True)
liver_gene_expr.fillna(0)
liver_meth_dna.fillna(0)

#Normalize and log tranform data for both sets
for col_name in col_names_exp[2:]:
  col_data = pd.to_numeric(np.log2(liver_gene_expr[col_name].values + 1))
  col_mean = np.mean(col_data)
  col_std = np.std(col_data)
  new_data = (col_data - col_mean)/(col_std)
  liver_gene_expr[col_name] = new_data
for col_name in col_names_meth[2:]:
  col_data = pd.to_numeric(np.log2(liver_meth_dna[col_name].values + 1))
  col_mean = np.mean(col_data)
  col_std = np.std(col_data)
  new_data = (col_data - col_mean)/(col_std)
  liver_meth_dna[col_name] = new_data

liver_gene_expr.head(n= 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


,Unnamed: 0,Label,X..100130426,X..100133144,X..100134869,X..10357,X..10431,X..136542,X..155060,X..26823,...,ZXDA.7789,ZXDB.158586,ZXDC.79364,ZYG11A.440590,ZYG11B.79699,ZYX.7791,ZZEF1.23140,ZZZ3.26009,psiTPTE22.387590,tAKR.389932
0,TCGA.2V.A95S.01,Primary Tumor,-0.25593,-0.476165,0.134622,0.066483,-0.283215,-0.067789,0.263113,1.214289,...,-0.101024,-0.836486,-0.61156,-0.096129,-1.126977,-0.055131,0.760924,-0.132179,-0.011595,-0.962797
1,TCGA.2Y.A9GS.01,Primary Tumor,-0.25593,2.268472,-0.189274,-0.522460,-1.766629,-0.067789,0.114870,2.651003,...,1.535443,0.324199,0.75920,0.835624,0.233890,0.845728,0.390778,1.258929,0.418190,1.458766


In [ ]:
liver_meth_dna.head(n = 20)

,Unnamed: 0,Label,cg00005740,cg00012529,cg00013196,cg00019366,cg00028598,cg00030627,cg00033551,cg00047815,...,rs7660805,rs7746156,rs798149,rs845016,rs877309,rs9292570,rs9363764,rs939290,rs951295,rs966367
0,TCGA.2V.A95S.01,Primary Tumor,0.159098,-0.828721,-0.806389,-0.963546,-0.616201,-0.873066,0.746242,-0.022433,...,0.999584,1.037542,0.954068,-0.123544,0.486531,0.104930,-0.190898,0.116703,0.252946,-1.258986
1,TCGA.2Y.A9GS.01,Primary Tumor,-0.492425,-0.747703,-0.455432,-0.497061,-0.745271,-0.438967,0.717706,-0.765602,...,-0.677251,-0.219410,0.958162,1.384457,0.507109,-1.490046,-0.095182,1.049926,-1.350460,0.273113
2,TCGA.2Y.A9GT.01,Primary Tumor,-0.360608,0.095973,-0.175213,-0.006948,-0.402580,-0.424622,-0.272663,-0.117828,...,-0.122326,-0.208594,-1.251790,1.424809,0.569711,-1.472544,-0.139657,1.063236,1.460783,1.530324
3,TCGA.2Y.A9GU.01,Primary Tumor,1.016130,1.048268,1.184712,0.168754,1.111800,1.083148,0.607155,0.890058,...,-0.038056,-0.138564,-1.260364,1.410127,-1.134521,0.139627,0.864928,1.062162,-1.392660,1.532507
4,TCGA.2Y.A9GV.01,Primary Tumor,0.296857,0.593994,0.094758,1.169701,-0.364758,0.222868,-1.661102,-0.113983,...,1.009447,1.033741,-1.259141,1.400324,0.634375,1.186729,-0.259052,0.010890,0.285969,-1.308029
5,TCGA.2Y.A9GW.01,Primary Tumor,-0.069750,-0.070568,-0.945250,0.657051,-0.005608,-0.081035,0.035320,-0.023371,...,0.060325,1.050311,-1.242252,-1.290586,-1.134259,1.197800,-0.100581,0.103337,1.467580,1.484892
6,TCGA.2Y.A9GX.01,Primary Tumor,-0.004305,-0.285313,0.272516,0.439574,0.280389,0.256453,0.059132,0.229998,...,-1.798643,-0.223802,-1.250119,-1.295826,0.460408,1.183660,0.850977,1.053215,0.300185,0.278903
7,TCGA.2Y.A9GY.01,Primary Tumor,-0.106606,-0.785025,-0.819248,-0.859750,-0.444648,-0.701739,0.765141,-0.838400,...,-0.508396,-0.081519,0.960086,0.593393,1.568633,0.134557,0.889647,0.308210,1.445359,1.536898
8,TCGA.2Y.A9GZ.01,Primary Tumor,-1.786311,-1.220929,-1.275734,-1.784193,-1.475135,-1.656015,-0.834236,-1.818103,...,0.351427,-1.740744,-0.129362,-1.312086,0.433762,1.187255,0.727637,0.455159,0.328524,0.292079
9,TCGA.2Y.A9H0.01,Primary Tumor,-1.720874,-1.504447,-0.994369,-1.156011,-1.028101,-1.296399,0.737024,-0.811471,...,-1.755452,-0.149602,0.957195,-1.315937,-1.139449,0.463880,-0.044830,0.078209,0.274847,0.225619


# PCA

PCA is likely required as there are 20033 parameters for the gene expression data and 5003 parameters for the methylation data. PCA is used for dimensionality reduction. 

In [66]:
# Using PCA
# turn into PCA
from sklearn.decomposition import PCA

exp_data = liver_gene_expr[col_names_exp[2:]].values

print(liver_gene_expr.apply(pd.isna).all())

#try 10 components or more
#exp_pca = PCA(n_components=10)
#exp_pc_data = exp_pca.fit_transform(exp_data)

#exp_pc_data.head()


AttributeError: ignored

# TensorBoard for Model Evaluation

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# %reload_ext tensorboard

%tensorboard --logdir /dir/

# or use %tensorboard --logdir /dir/

# Solid Tissue Normal vs Primary Tumour Classifier


In [ ]:
# Need to make data and labels variables 

import tensorflow as tf

np.random.seed(129)
tf.random.set_seed(129)

# JUST reference info below here

# Support Vector Machine

In [ ]:
## Encode the labels in 1 hot encoding
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

# Run a SVM
# https://scikit-learn.org/stable/modules/svm.html
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
plot_confusion_matrix(clf, X_test, y_test)  
plt.show()
plot_confusion_matrix(clf, data, labels) 

# Neural Network


In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.33, random_state=42)
# create model

model = Sequential()
model.add(Dense(2,  activation='sigmoid', input_shape=(3,),  name="bottleneck"))
model.add(Dense(3,  activation='softmax'))


#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs = 150, batch_size = 4)

In [ ]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
encoder_p = Model(model.input, model.get_layer('bottleneck').output)

nn_enc = encoder_p.predict(data)  # bottleneck representation

# plot
for tissue in tissues:
    tissue_idxs = np.where(df['tissue'].values == tissue)
    sc = plt.scatter(nn_enc[tissue_idxs, 0], nn_enc[tissue_idxs, 1], s=60, label=tissue, marker='o', alpha=0.8)
    
ax.set_xlabel('AE 0')
ax.set_ylabel('AE 1')
plt.legend()
plt.title('AE of age, years smoked, BMI')
plt.show()


df['AE0'] = nn_enc[:, 0]
df['AE1'] = nn_enc[:, 1]

# plot each of the features on the plot! We'll relate these later 
cmaps = ['BuGn', 'PuRd', 'YlOrBr']
for i, s in enumerate(['bmi', 'smoke', 'age']):    
    sc = plt.scatter(df['AE0'], df['AE1'], s=60, label=s, c=df[s], marker='o', alpha=0.8, cmap=cmaps[i])

    ax.set_xlabel('BMI')
    ax.set_ylabel('Smoked years')
    plt.colorbar()
    plt.title(f'AE BMI, Smoked years and Age, colour: {s}')
    plt.show()

# Check Metrics of Models

In [ ]:
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical

predictions = model.predict(data)
prediction_ = np.argmax(predictions, axis = 1)
prediction_ = encoder.inverse_transform(prediction_)

for i, j in zip(prediction_ , labels):
    print( " the nn predict {}, and the tumour stage is {}".format(i,j))
confusion_matrix(prediction_, labels)


# New Section